<a href="https://colab.research.google.com/github/TheCaveOfAdullam/study3/blob/main/1005Test7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import LabelEncoder

In [3]:
# 기본 경로 설정
base_dir = '/content/drive/MyDrive/ship_motor10'
categories = ['normal', 'fault_BB', 'fault_RI', 'fault_SM']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 로드 및 전처리 함수 정의
class VibrationDataset(Dataset):
    def __init__(self, base_dir, split, categories, transform=None):
        self.X = []
        self.y = []
        self.transform = transform
        split_dir = os.path.join(base_dir, split)
        for category in categories:
            category_dir = os.path.join(split_dir, category)
            files = os.listdir(category_dir)
            for file in files:
                file_path = os.path.join(category_dir, file)
                data = pd.read_csv(file_path, header=None).values
                data = pd.to_numeric(data.flatten(), errors='coerce').reshape(-1, data.shape[1])
                data = np.nan_to_num(data).astype('float32')  # NaN 값을 0으로 대체하고, float32로 변환
                self.X.append(data)
                self.y.append(category)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        X = X.reshape(-1)  # 마지막 차원을 시퀀스에 병합하여 [sequence_length]로 변환
        return torch.tensor(X, dtype=torch.float32), y

# 레이블 인코딩
label_encoder = LabelEncoder()

# 데이터셋 준비
train_dataset = VibrationDataset(base_dir, 'train', categories)
val_dataset = VibrationDataset(base_dir, 'validation', categories)
test_dataset = VibrationDataset(base_dir, 'test', categories)

# 레이블 인코딩 및 원-핫 인코딩
y_train_encoded = label_encoder.fit_transform([y for _, y in train_dataset])
y_val_encoded = label_encoder.transform([y for _, y in val_dataset])
y_test_encoded = label_encoder.transform([y for _, y in test_dataset])

# 데이터셋에 레이블 추가
train_dataset.y = y_train_encoded
val_dataset.y = y_val_encoded
test_dataset.y = y_test_encoded

# 데이터 로더
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [82]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=16, stride=16)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=3, stride=1)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1)
        self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        conv1_output_size = (24002 - 16) // 16 + 1
        pool1_output_size = conv1_output_size // 2
        conv2_output_size = (pool1_output_size - 3) // 1 + 1
        conv3_output_size = (conv2_output_size - 5) // 1 + 1
        conv4_output_size = (conv3_output_size - 5) // 1 + 1
        pool2_output_size = conv4_output_size // 2

        self.fc1 = nn.Linear(128 * pool2_output_size, 5000)
        self.fc2 = nn.Linear(5000, 1000)
        self.fc3 = nn.Linear(1000, len(categories))

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.conv3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = torch.relu(x)

        # 1x1 컨볼루션 적용해서 차원 맞추기
        if hasattr(self, 'adjust_conv4'):
            x = self.adjust_conv4(x)

        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
model = CNNModel().to(device)

In [83]:
# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [84]:
# 모델 훈련 함수
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss, val_accuracy = evaluate_model(model, val_loader, criterion)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/total:.4f}, Accuracy: {100 * correct/total:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

In [85]:
# 모델 평가
def evaluate_model(model, loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return val_loss / total, 100 * correct / total

In [86]:
# 매그니튜드 기반 프루닝 함수
def prune_by_magnitude(model, threshold):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
            weight_abs = torch.abs(module.weight.data)
            mask = weight_abs > threshold
            module.weight.data[~mask] = 0
    print(f'Magnitude-based pruning with threshold: {threshold} applied.')

# 1차 테일러 전개 기반 프루닝 함수
def prune_by_taylor(model, threshold):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d) or isinstance(module, nn.Linear):
            importance = torch.abs(module.weight * module.weight.grad)
            mask = importance > threshold
            with torch.no_grad():
                module.weight[~mask] = 0
    print(f'Taylor expansion-based pruning with threshold: {threshold} applied.')

In [87]:
def prune_filters_with_zero_weight_ratio(model, threshold=0.9):
    """필터의 가중치가 90% 이상 0일 경우 필터를 제거하고, 다음 레이어의 입력 채널을 맞춤."""

    layers_to_prune = []  # 나중에 교체할 레이어를 기록해두기 위한 리스트
    prev_out_channels = None  # 이전 레이어의 출력 채널 수를 저장할 변수

    for name, module in list(model.named_modules()):
        if isinstance(module, nn.Conv1d):
            weight = module.weight.detach().cpu().numpy()
            zero_ratio_per_filter = np.mean(weight == 0, axis=(1, 2))  # 필터별로 0의 비율 계산

            # 0의 비율이 threshold 이상인 필터의 인덱스를 추출
            prune_indices = [i for i, ratio in enumerate(zero_ratio_per_filter) if ratio >= threshold]

            if prune_indices:
                # 필터 제거 후 새로운 가중치 텐서 생성
                new_weight = torch.cat([w.unsqueeze(0) for i, w in enumerate(module.weight) if i not in prune_indices], dim=0)
                new_bias = torch.cat([b.unsqueeze(0) for i, b in enumerate(module.bias) if i not in prune_indices], dim=0)

                # 새로운 레이어 정의
                new_layer = nn.Conv1d(
                    in_channels=module.in_channels,
                    out_channels=new_weight.size(0),  # 남은 필터 수
                    kernel_size=module.kernel_size,
                    stride=module.stride,
                    padding=module.padding
                )
                new_layer.weight = nn.Parameter(new_weight)
                new_layer.bias = nn.Parameter(new_bias)

                # 프루닝 후 레이어 출력 채널 수를 저장
                prev_out_channels = new_weight.size(0)

                # 레이어 교체를 기록
                layers_to_prune.append((name, new_layer))

        # 다음 레이어가 Conv1d일 경우, 입력 채널을 프루닝된 출력 채널로 수정
        if prev_out_channels and isinstance(module, nn.Conv1d) and module.in_channels != prev_out_channels:
            new_layer = nn.Conv1d(
                in_channels=prev_out_channels,  # 프루닝된 레이어의 출력 채널을 입력 채널로 설정
                out_channels=module.out_channels,
                kernel_size=module.kernel_size,
                stride=module.stride,
                padding=module.padding
            )
            new_layer.weight = module.weight
            new_layer.bias = module.bias

            # 레이어 교체를 기록
            layers_to_prune.append((name, new_layer))

    # 레이어 교체 (모델을 순회한 후에 교체해야 함)
    for name, new_layer in layers_to_prune:
        parent_module, layer_name = find_parent_module(model, name)
        setattr(parent_module, layer_name, new_layer)

    print(f'Filters pruned based on zero weight ratio (threshold={threshold}).')
    return model


def find_parent_module(model, layer_name):
    """모델에서 주어진 레이어 이름을 가진 모듈의 부모 모듈과 해당 모듈의 이름을 찾는 함수."""
    parts = layer_name.split(".")
    current_module = model
    for part in parts[:-1]:
        current_module = getattr(current_module, part)
    return current_module, parts[-1]

In [88]:
# 1. 초기 7 에폭 훈련
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=3)

Epoch [1/3], Loss: 0.0391, Accuracy: 50.00%, Val Loss: 0.0393, Val Accuracy: 50.00%
Epoch [2/3], Loss: 0.0390, Accuracy: 50.00%, Val Loss: 0.0392, Val Accuracy: 50.00%
Epoch [3/3], Loss: 0.0313, Accuracy: 60.17%, Val Loss: 0.0055, Val Accuracy: 99.22%


In [89]:
# 2. 매그니튜드 기반 프루닝 적용
magnitude_threshold = 0.1
prune_by_magnitude(model, magnitude_threshold)

Magnitude-based pruning with threshold: 0.1 applied.


In [90]:
# 3. 매그니튜드 기반 프루닝 후 파인튜닝
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=1)

Epoch [1/1], Loss: 0.0392, Accuracy: 50.00%, Val Loss: 0.0398, Val Accuracy: 50.00%


In [91]:
# 4. 1차 테일러 전개 기반 프루닝 적용
taylor_threshold = 0.001
prune_by_taylor(model, taylor_threshold)

Taylor expansion-based pruning with threshold: 0.001 applied.


In [92]:
# 5. 테일러 전개 기반 프루닝 후 파인튜닝
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=3)

Epoch [1/3], Loss: 0.0392, Accuracy: 50.00%, Val Loss: 0.0393, Val Accuracy: 50.00%
Epoch [2/3], Loss: 0.0389, Accuracy: 50.00%, Val Loss: 0.0393, Val Accuracy: 50.00%
Epoch [3/3], Loss: 0.0389, Accuracy: 50.00%, Val Loss: 0.0393, Val Accuracy: 50.00%


In [93]:
# 6. 구조적 프루닝 (90% 이상의 필터 제거)
model = prune_filters_with_zero_weight_ratio(model, threshold=0.7)

Filters pruned based on zero weight ratio (threshold=0.7).


In [94]:
# 7. 구조적 프루닝 후 파인튜닝
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=3)

Epoch [1/3], Loss: 0.0389, Accuracy: 50.00%, Val Loss: 0.0393, Val Accuracy: 50.00%
Epoch [2/3], Loss: 0.0389, Accuracy: 50.00%, Val Loss: 0.0393, Val Accuracy: 50.00%
Epoch [3/3], Loss: 0.0389, Accuracy: 50.00%, Val Loss: 0.0393, Val Accuracy: 50.00%


In [95]:
# 테스트 성능 확인
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.0393, Test Accuracy: 50.00%


In [96]:
# 모델 저장
torch.save(model.state_dict(), 'model.pth')

In [97]:
# 모델 사이즈 확인
model_size = os.path.getsize('model.pth') / (1024 * 1024)
print(f"Model Size: {model_size:.2f} MB")

Model Size: 922.66 MB


In [102]:
# 파라미터 수 확인
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Total number of trainable parameters: {total_params}")

Total number of trainable parameters: 241868660


In [99]:
def check_zero_ratio(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            weight = module.weight.detach().cpu().numpy()
            zero_ratio_per_filter = np.mean(weight == 0, axis=(1, 2))
            print(f"Layer: {name}, Zero ratio per filter: {zero_ratio_per_filter}")

# 프루닝 후 0 가중치 비율 확인
check_zero_ratio(model)

Layer: conv1, Zero ratio per filter: [0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Layer: conv2, Zero ratio per filter: [1.         0.51041667 0.234375   0.234375   0.23958333 0.328125
 0.234375   0.28125    0.234375   1.         0.41666667 0.28125
 0.28125    0.3125     1.         0.25520833 0.23958333 0.234375
 0.27083333 0.296875   0.234375   0.234375   0.28125    0.234375
 0.234375   0.234375   0.23958333 0.328125   0.28125    0.234375
 0.48958333 0.234375  ]
Layer: conv3, Zero ratio per filter: [0.23125 1.      0.38125 0.10625 0.10625 0.15    1.      0.1125  1.
 0.10625 1.      0.1375  0.125   1.      0.5     0.14375 0.10625 0.425
 1.      0.10625 0.23125 0.1375  0.15625 0.15625 0.21875 0.59375 1.
 0.10625 1.      0.1375  1.      0.23125 0.10625 1.      0.1375  0.125
 0.10625 0.10625 0.15625 0.10625 0.10625 0.1875  0.10625 0.375   0.